# Input Library

In [3]:
import tensorflow as tf
from tensorflow.keras.layers import Flatten, Dense, Input

# Model Functional API & Sequential API

In [4]:
# seq_model = Sequential([
#     Flatten(input_shape=(28, 28)),
#     Dense(128, activation='relu'),
#     Dense(10, activation='softmax')
# ])

# TensorFlow Functional API
input = Input(shape=(28, 28))
x = Flatten()(input)
x = Dense(128, activation='relu')(x)
predictions = Dense(10, activation='softmax')(x)

model = tf.keras.Model(inputs=input, outputs=predictions)

# Custom Loss Function

In [ ]:
def custom_loss(y_true, y_pred):    